In [1]:
import numpy as np
import scipy.stats as st
import logging
from enum import Enum
from typing import List

In [2]:
# Configuration of the simulation
# N_DIMS = 2
# N_AGENTS = 10**3
# STEPS = 5 * 10**3
# LOW, HIGH = -25, 25
# MIN_VEL, MAX_VEL = -5, 5

# Test functions --> change target_func for using each of them

def sphere(vector):
    return np.sum(np.power(vector, 2))
    
def rastrigin(vector, A=10):
    return A + np.sum(np.power(vector, 2) - A * np.cos(2*np.math.pi*vector))

def rosenbrock(vector, A=100, B=20):
    return np.math.exp(-np.sum(np.array([(A*(vector[i+1]-vector[i]**2)**2 + (1-vector[i])**2)/B for i in range(len(vector) - 1)])))

target_func = lambda vector: sphere(vector)  # OBJECTIVE FUNCTION USED BY PARTICLES